In [2]:
import pandas as pd
from bs4 import Tag
from bs4 import BeautifulSoup
import requests
import time

In [19]:
#Site link to get data from
main_site = "https://www.studentcrowd.com/"

#Hub page to get links to university halls
main_link = "https://www.studentcrowd.com/best-halls-l1001909-s1008497-the_university_of_warwick-coventry"

main_page = requests.get(main_link)
main_soup = BeautifulSoup(main_page.content, 'html.parser')

In [20]:
links = main_soup.find_all('a')
url_list = []
for i in range(len(links)):
  link = links[i]          # get the first link in the entire page
  url = link['href']

  if url.startswith("hall"):
    url_list.append(url)    # get value of the href attribute

#url_list is a list containing all urls for main_site

In [21]:
url_list

['hall-l1001909-s1052584-city-point-coventry#booking',
 'hall-l1001909-s1045417-merlin-point-coventry#booking',
 'hall-l1001909-s1054956-urban-student-village-coventry#booking',
 'hall-l1001909-s1049089-market-way-coventry#booking',
 'hall-l1001909-s1052540-gulson-gardens-coventry#booking',
 'hall-l1001909-s1045635-abbey-house-coventry#booking',
 'hall-l1001909-s1045497-paradise-student-village-coventry#booking',
 'hall-l1001909-s1054709-canvas-coventry-coventry#booking',
 'hall-l1001909-s1052614-iq-weaver-place-coventry#booking',
 'hall-l1001909-s1049425-mercia-lodge-coventry#booking',
 'hall-l1001909-s1054751-infinity-coventry#booking',
 'hall-l1001909-s1045503-apollo-house-coventry#booking',
 'hall-l1001909-s1045615-julian-court-coventry#booking',
 'hall-l1001909-s1052540-gulson-gardens-coventry#booking',
 'hall-l1001909-s1045635-abbey-house-coventry#booking',
 'hall-l1001909-s1052584-city-point-coventry#booking',
 'hall-l1001909-s1045615-julian-court-coventry#booking',
 'hall-l1001

In [5]:
#Open up individual accommadation review site a

a = 0

review_page = requests.get(main_site + url_list[a])
review_soup = BeautifulSoup(review_page.content, 'html.parser')
#review_soup.prettify

In [6]:
#Get name of accommodation

review_soup.select(".cb")[0].get_text()

'City Point'

In [7]:
#Each review consists of 7 stars, so we group the stars based on the review it is attached to
#NOTE that 1st review is the 'most helpful' review and so is duplicated. That is why i starts at 7.
def get_stars(soup):
    star_list = []
    for j in range(7):
        star_list.append([])

    for i in range(7,len(soup.select(".stars"))):
        #Set star = ith star
        star = int(soup.select(".stars")[i]['class'][1][-1])
        star_list[i%7].append(star)
    
    return star_list

sample_star_list = get_stars(review_soup)
sample_star_list

[[5, 5, 5], [5, 5, 5], [5, 5, 5], [5, 5, 5], [5, 4, 5], [4, 4, 4], [5, 5, 5]]

In [8]:
#Get list of all reviews without replies

def get_reviews(soup):
    review_list = []
    for i in range(7,len(soup.select(".cb")),2):
      review_text = soup.select(".cb")[i].get_text()
      #Every review has the string 'more...' at the end. Drop last 7 characters from every text
      review_list.append(review_text[:-7])
    
    #Get list of all replies
    reply_list = []
    for i in range(1,len(soup.select(".js-review-reply"))):
      review_text = soup.select(".js-review-reply")[i].get_text()
      #Every review has the string 'more...' at the end. Drop last 7 characters from every text
      reply_list.append(review_text[:-7])
    
    #Find the differences using sets
    review_list = list(set(review_list) -  set(reply_list))
    return review_list

sample_review_list = get_reviews(review_soup)
sample_review_list

['Really recommand City Point！It has a perfect location and you can get whatever you want from the Coventry city centre. The environment of the apartments are peaceful and tidy. The reception and security here are responsible and friendly for any queries.',
 'Very good commendation. I would highly recommend City Point to those who are planing to study in Coventry and Warwick. The location of CP is perfect, it takes about 3 mins to get to the Coventry city centre where I can get anything I want. The reception and security of CP are really friendly and helpful！Really good choice！',
 'Love staying in City Point! Living in City Point has really lightened my student life in Coventry. The ensuite room is comfortable, the flat common area is large. We have 5 people in the flat, and the common room suits our need for cooking and leisure. Besides, the reception staffs are helpful and caring. Very recommend to students!']

In [31]:
#99999999999999999999999999999999999999999999999999999999999999999999999999999999

def list_to_dataframe(name_list, star_list, review_list):
    df_output = pd.DataFrame(list(zip(name_list,
                                      star_list[0], star_list[1],
                                      star_list[2], star_list[3],
                                      star_list[4], star_list[5],
                                      star_list[6], review_list)),
                             columns=['Accom', 'Value_for_money',
                                      'Location','Property_management',
                                      'Cleaning', 'Social_spaces', 
                                      'Social_experience',
                                      'Internet', 'Review'])
    return df_output

In [28]:
def scrape_page(link):
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #Scrape star data and review data
    star_list = get_stars(soup)
    review_list = get_reviews(soup)
    
    #Get name of accom
    name = soup.select(".cb")[0].get_text()
    
    #Convert to list
    output_list = [name, star_list, review_list]
    
    
    
    return output_list
scrape_page(main_site + url_list[a])

['City Point',
 [[5, 5, 5], [5, 5, 5], [5, 5, 5], [5, 5, 5], [5, 4, 5], [4, 4, 4], [5, 5, 5]],
 ['Really recommand City Point！It has a perfect location and you can get whatever you want from the Coventry city centre. The environment of the apartments are peaceful and tidy. The reception and security here are responsible and friendly for any queries.',
  'Very good commendation. I would highly recommend City Point to those who are planing to study in Coventry and Warwick. The location of CP is perfect, it takes about 3 mins to get to the Coventry city centre where I can get anything I want. The reception and security of CP are really friendly and helpful！Really good choice！',
  'Love staying in City Point! Living in City Point has really lightened my student life in Coventry. The ensuite room is comfortable, the flat common area is large. We have 5 people in the flat, and the common room suits our need for cooking and leisure. Besides, the reception staffs are helpful and caring. Very r

In [35]:
#Takes main_link to the homepage containing various accommodation sites and url homepage
#For studentcrowd, use main_site = "https://www.studentcrowd.com/"

def crawl_homepage(link, site):
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #Find links to individual accom sites
    links = soup.find_all('a')
    
    #url_list is a list containing all urls for main_site
    url_list = []
    for i in range(len(links)):
        link = links[i]          # get the first link in the entire page
        url = link['href']

        if url.startswith("hall"):
            url_list.append(url)    # get value of the href attribute
    
    #List to hold names
    final_name_list = []
    
    #Lists to store all star and review data
    final_star_list = []
    for i in range(7):
        final_star_list.append([])
    final_review_list = []

    #Loop across different sites
    for i in range(len(url_list)):
        output_list = scrape_page(site + url_list[i])
        
        #Iterate create name list
        name = output_list[0]
        for i in range(len(output_list[2])):
            final_name_list.append(name)
        
        #Iterate over each star list to add lists together
        for j in range(7):
            final_star_list[j] = final_star_list[j] + output_list[1][j]
        
        final_review_list = final_review_list + output_list[2]
        
        #Delay
        time.sleep(5)
        
        
    #Output everything into a dataframe
    df_output = list_to_dataframe(final_name_list, final_star_list, final_review_list)
    return df_output

In [47]:
#Hub page to get links to university halls
main_link = "https://www.studentcrowd.com/best-halls-l1001909-s1008497-the_university_of_warwick-coventry"

#Site link to get data from
main_site = "https://www.studentcrowd.com/"

df_data_raw = crawl_homepage(main_link, main_site)
df_data = df_data_raw

In [51]:
df_data.rename(columns = {'Accom Name': 'Accom'}, inplace=True)
df_data

,Accom,Value_for_money,Location,Property_management,Cleaning,Social_spaces,Social_experience,Internet,Review
0,iQ Weaver Place,4,5,5,4,5,5,5,IQ weaver place has been an amazing place to r...
1,Julian Court,5,5,5,5,3,3,5,I am living in classic studio with a window fa...
2,Julian Court,5,4,5,5,4,4,5,I have a very nice experience staying in Julia...
3,Julian Court,5,4,5,5,4,4,5,"As a self-contained studio apartment, Julian C..."
4,Julian Court,5,5,5,5,5,5,5,"I like the young lady in reception, she is alw..."
...,...,...,...,...,...,...,...,...,...
672,Most helpful review,4,5,5,5,4,4,5,Student Review:\n1\n/5By member1132949 On \n19...
673,Most helpful review,4,4,2,5,3,4,3,Student Review:\n4\n/5By member935348 On \n20t...
674,Most helpful review,4,5,5,5,4,4,5,Student Review:\n2\n/5By member937829 On \n13th
675,Most helpful review,3,4,5,5,5,4,4,Student Review:\n2\n/5By member635390 On \n3rd...


In [52]:
df_data.drop( df_data[df_data['Accom'] == "Most helpful review"].index, inplace=True )
df_data

,Accom,Value_for_money,Location,Property_management,Cleaning,Social_spaces,Social_experience,Internet,Review
0,iQ Weaver Place,4,5,5,4,5,5,5,IQ weaver place has been an amazing place to r...
1,Julian Court,5,5,5,5,3,3,5,I am living in classic studio with a window fa...
2,Julian Court,5,4,5,5,4,4,5,I have a very nice experience staying in Julia...
3,Julian Court,5,4,5,5,4,4,5,"As a self-contained studio apartment, Julian C..."
4,Julian Court,5,5,5,5,5,5,5,"I like the young lady in reception, she is alw..."
...,...,...,...,...,...,...,...,...,...
91,Paradise Student Village,5,4,3,2,3,3,2,Best accommodation I have been too
92,Paradise Student Village,1,2,1,1,1,1,3,"During lockdown, students were forced to pay f..."
93,Paradise Student Village,3,4,4,5,4,1,1,"Most of the experience is quite good. However,..."
94,Paradise Student Village,1,2,2,3,2,2,4,Good management and security on site. Bikes ar...


In [55]:
df_data[['Accom', 'Review']].to_csv(r'AccomReviewSample.csv')